# 16 1 5

Adding in expected secondary position, which will be used to find the observed secondary lobe closest to the expected secondary position. 

PS3 takes in the following data fields: 
- *TID* — Target ID. If the Target ID is a “D”, then PS3 will process this file, otherwise it will be ignored.
This allows a master database to contain, for instance, single stars used for deconvolution, slit mask
or drift calibrations, etc.
- *DSFN* — Double star (FITS cube) file name.
- *RSFN* — Reference star (FITS) cube file name.
- *ThetaC* — Calculated (input) estimated, predicted, or last catalog reported double star position angle.
- *RhoC* — Calculated (input) estimated, predicted, or last catalog reported double star separation.
- *Delta* — Camera Angle (degrees). Camera orientation angle with respect to the sky.
- *E* — Plate scale (arc seconds / pixel).

For all the FITS data I am working with from April 2014 Kitt Peak run:
- *Delta* = 0 [degrees]
- *E* = 0.01166 [arc seconds / pixel]


In [ ]:
# Using calculated centroid data from end of previous notebook along 
#  with calibration and expected position data from run log to calculate 
#  observed secondary location.

In [ ]:
# Import Modules
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy.fftpack import fft2,ifft2,fftshift
from speckle_fns import fits_import, circ_filter1
from speckle_fns import deconv0,deconv1, postprocess
from speckle_fns import filter_lpf, filter_hpf, filter_interference
from speckle_fns import fits_view
import sys
import cv2
import math

%matplotlib inline

In [ ]:
# Centroid values from end of 15-12-28 notebook:
centroids = np.zeros((5,2,2))

centroids[0] = [[ 403.24772626, 405.91946966],[ 108.75227374, 106.08053034]]
centroids[1] = [[ 220.00967484, 255.04554408],[ 291.99032516, 256.95445592]]
centroids[2] = [[ 469.02557551, 506.57034716],[ 42.97582522, 5.06646098]]
centroids[3] = [[ 225.11283002, 274.88879163],[ 286.88716998, 237.11120837]]
centroids[4] = [[ 282.11470187, 275.93283984],[ 229.88529813, 236.06716016]]

# Expected positions of each pair from "KP 4-14 Run Log.xlsx"
thetaCs = np.array((204,63,32,305,124))
rhoCs = np.array((44.7,4.9,0.3,0.5,0.4))

# Calibration values from "KP 4-14 Run Log.xlsx"
delta = 0
E = 0.01166

In [ ]:
# Select which set of data to work with

k = 0

centroid = centroids[k]
thetaC = thetaCs[k]
rhoC = rhoCs[k]

In [ ]:
## Displaying calculated centroid values along with expected secondary star locations

# Create blank image
img = np.zeros((512,512,3), np.uint8)

# Parameters for marking objects
circle_radius = 11 # Radius of circle around location
circle_border = 1 # Border of circle around location
color_centroid = (255,0,0) # Color of centroid marker
color_secondaryC = (0,255,0) # Color of expected secondary star location

# Mark centroid locations on image
cv2.circle(img, (centroid[0,0].astype(np.uint16),centroid[0,1].astype(np.uint16)), circle_radius, color_centroid, circle_border)
cv2.circle(img, (centroid[1,0].astype(np.uint16),centroid[1,1].astype(np.uint16)), circle_radius, color_centroid, circle_border)

# Calculate pixel locations of expected secondary location
thetaC_camera = thetaC-delta
rhoC_camera = rhoC/E

print("thetaC_camera=",thetaC_camera, "degrees")
print("rhoC_camera=",rhoC_camera, "pixels")

plt.figure(figsize=(10,10))
plt.imshow(img)